In [1]:
# ===== CELL 1: MEMORY SETUP - RUN THIS FIRST =====
import torch
import gc
import os

# Clear any existing GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    
# Set memory allocation strategy
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Monitor GPU memory
def print_gpu_memory():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1e9
        reserved = torch.cuda.memory_reserved(0) / 1e9
        total = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"GPU Memory: {allocated:.2f}GB allocated / {reserved:.2f}GB reserved / {total:.2f}GB total")
    
print_gpu_memory()
print("Memory setup complete!")

GPU Memory: 0.00GB allocated / 0.00GB reserved / 85.03GB total
Memory setup complete!


In [ ]:
# Fixed DARTS Implementation for Pancreatic Tumor Classification
# Addresses: operation collapse, proper bi-level optimization, entropy regularization

import os
import random
import numpy as np
from pathlib import Path
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image

from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import xgboost as xgb
import joblib

# ==================== CONFIGURATION ====================
SEED = 42

# Dataset paths - adjust based on your setup
DATA_ROOT = "./dataset"  # Change to your dataset location
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
TEST_DIR = os.path.join(DATA_ROOT, "test")

# Verify dataset exists
if not os.path.exists(DATA_ROOT):
    print(f"⚠️  WARNING: Dataset not found at {DATA_ROOT}")
    print("Please upload your dataset or update DATA_ROOT path")
else:
    print(f"✓ Dataset found at {DATA_ROOT}")

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# GPU Memory optimization
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    # Enable memory efficient settings
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device.type == 'cuda':
    torch.cuda.manual_seed_all(SEED)

# ==================== PRIMITIVES ====================
PRIMITIVES = [
    "sep_conv_3x3",
    "sep_conv_5x5",
    "dil_conv_3x3",
    "avg_pool_3x3",
    "max_pool_3x3",
    "skip_connect",
]

# ==================== OPERATIONS ====================
class SepConv(nn.Module):
    def __init__(self, C, kernel_size):
        super().__init__()
        padding = kernel_size // 2
        self.op = nn.Sequential(
            nn.ReLU(inplace=False),
            nn.Conv2d(C, C, kernel_size=kernel_size, padding=padding, groups=C, bias=False),
            nn.Conv2d(C, C, kernel_size=1, bias=False),
            nn.BatchNorm2d(C),
        )
    
    def forward(self, x):
        return self.op(x)

class DilConv(nn.Module):
    def __init__(self, C, kernel_size, dilation):
        super().__init__()
        padding = dilation * (kernel_size - 1) // 2
        self.op = nn.Sequential(
            nn.ReLU(inplace=False),
            nn.Conv2d(C, C, kernel_size=kernel_size, padding=padding, 
                     dilation=dilation, bias=False),
            nn.BatchNorm2d(C),
        )
    
    def forward(self, x):
        return self.op(x)

class PoolBN(nn.Module):
    def __init__(self, C, pool_type='max'):
        super().__init__()
        if pool_type == 'max':
            self.pool = nn.MaxPool2d(3, stride=1, padding=1)
        else:
            self.pool = nn.AvgPool2d(3, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(C)
    
    def forward(self, x):
        return self.bn(self.pool(x))

class Identity(nn.Module):
    def forward(self, x):
        return x

def get_op(name, C):
    """Get operation by name"""
    if name == "sep_conv_3x3":
        return SepConv(C, 3)
    elif name == "sep_conv_5x5":
        return SepConv(C, 5)
    elif name == "dil_conv_3x3":
        return DilConv(C, 3, 2)
    elif name == "avg_pool_3x3":
        return PoolBN(C, 'avg')
    elif name == "max_pool_3x3":
        return PoolBN(C, 'max')
    elif name == "skip_connect":
        return Identity()
    else:
        raise ValueError(f"Unknown operation: {name}")

# ==================== MIXED OPERATION ====================
class MixedOp(nn.Module):
    """Weighted combination of operations"""
    def __init__(self, C):
        super().__init__()
        self.ops = nn.ModuleList([get_op(name, C) for name in PRIMITIVES])
    
    def forward(self, x, weights):
        """
        Args:
            x: input tensor
            weights: softmax weights for operations
        """
        return sum(w * op(x) for w, op in zip(weights, self.ops))

# ==================== CELL ====================
class Cell(nn.Module):
    """DARTS cell with mixed operations"""
    def __init__(self, C, num_nodes=4):
        super().__init__()
        self.num_nodes = num_nodes
        
        # Create mixed ops for each intermediate node
        # Each node takes input from all previous nodes
        self.ops = nn.ModuleList()
        for i in range(num_nodes):
            for j in range(i + 2):  # i+2 because we have 2 input nodes
                self.ops.append(MixedOp(C))
    
    def forward(self, x, alphas):
        """
        Args:
            x: input tensor
            alphas: architecture parameters
        """
        states = [x, x]  # Two initial states
        offset = 0
        
        for i in range(self.num_nodes):
            s = 0
            for j, h in enumerate(states):
                weights = F.softmax(alphas[offset + j], dim=-1)
                s = s + self.ops[offset + j](h, weights)
            offset += len(states)
            states.append(s)
        
        # Concatenate all intermediate nodes
        return sum(states[-self.num_nodes:]) / self.num_nodes

# ==================== SEARCH NETWORK ====================
class SearchNetwork(nn.Module):
    def __init__(self, C=16, num_cells=8, num_nodes=4, num_classes=2):
        super().__init__()
        self.C = C
        self.num_cells = num_cells
        self.num_nodes = num_nodes
        
        # Stem
        self.stem = nn.Sequential(
            nn.Conv2d(3, C, 3, padding=1, bias=False),
            nn.BatchNorm2d(C),
            nn.ReLU()
        )
        
        # Cells
        self.cells = nn.ModuleList([Cell(C, num_nodes) for _ in range(num_cells)])
        
        # Classifier
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(C, num_classes)
        
        # Architecture parameters
        self.num_edges = sum(range(2, num_nodes + 2))
        self.alphas = nn.Parameter(
            torch.randn(self.num_edges, len(PRIMITIVES)) * 1e-3
        )
    
    def forward(self, x):
        s = self.stem(x)
        
        for cell in self.cells:
            s = cell(s, self.alphas)
        
        out = self.global_pool(s).view(s.size(0), -1)
        logits = self.classifier(out)
        return logits
    
    def genotype(self):
        """Derive genotype from architecture parameters"""
        gene = []
        
        alphas_np = self.alphas.detach().cpu().numpy()
        for alpha in alphas_np:
            # Get operation with maximum weight
            op_idx = np.argmax(alpha)
            gene.append(PRIMITIVES[op_idx])
        
        return gene


# ==================== DATASET ====================
class PancreaticDataset(Dataset):
    def __init__(self, root_dir, class_to_idx=None, img_size=224, augment=False):
        self.root = Path(root_dir)
        
        # Get class folders
        classes = sorted([d.name for d in self.root.iterdir() 
                         if d.is_dir() and not d.name.startswith('.')])
        
        if class_to_idx is None:
            self.class_to_idx = {c: i for i, c in enumerate(classes)}
        else:
            self.class_to_idx = class_to_idx
        
        # Collect samples
        self.samples = []
        for cls in classes:
            cls_dir = self.root / cls
            for img_path in cls_dir.glob("*"):
                if img_path.suffix.lower() in [".png", ".jpg", ".jpeg", ".bmp"]:
                    self.samples.append((str(img_path), self.class_to_idx[cls]))
        
        # Transforms
        if augment:
            self.transform = transforms.Compose([
                transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(15),
                transforms.ColorJitter(brightness=0.2, contrast=0.2),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        img = self.transform(img)
        return img, label

# ==================== ARCHITECT (for alpha optimization) ====================
class Architect:
    """Handles architecture parameter optimization"""
    def __init__(self, model, args):
        self.model = model
        self.optimizer = torch.optim.Adam(
            [model.alphas],
            lr=args['arch_lr'],
            betas=(0.5, 0.999),
            weight_decay=args['arch_weight_decay']
        )
    
    def step(self, x_val, y_val):
        """Update architecture parameters"""
        self.optimizer.zero_grad()
        logits = self.model(x_val)
        loss = F.cross_entropy(logits, y_val)
        
        # Add entropy regularization to prevent collapse
        alphas_softmax = F.softmax(self.model.alphas, dim=-1)
        entropy = -(alphas_softmax * torch.log(alphas_softmax + 1e-8)).sum(dim=-1).mean()
        
        # We want to maximize entropy (prevent one-hot), so subtract it
        total_loss = loss - 0.01 * entropy
        
        total_loss.backward()
        self.optimizer.step()
        
        return loss.item(), entropy.item()

# ==================== TRAINING ====================
def train_search(model, train_loader, val_loader, architect, epochs=10):
    """Train search network with bi-level optimization"""
    
    # Weight optimizer
    w_optimizer = torch.optim.SGD(
        [p for n, p in model.named_parameters() if 'alphas' not in n],
        lr=0.025,
        momentum=0.9,
        weight_decay=3e-4
    )
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        w_optimizer, T_max=epochs, eta_min=1e-4
    )
    
    val_iter = iter(val_loader)
    
    # Enable gradient checkpointing to save memory
    torch.cuda.empty_cache()
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        arch_loss_sum = 0.0
        entropy_sum = 0.0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        
        for step, (x_train, y_train) in enumerate(pbar):
            # Move to device
            x_train, y_train = x_train.to(device, non_blocking=True), y_train.to(device, non_blocking=True)
            
            # Get validation batch
            try:
                x_val, y_val = next(val_iter)
            except StopIteration:
                val_iter = iter(val_loader)
                x_val, y_val = next(val_iter)
            
            x_val, y_val = x_val.to(device, non_blocking=True), y_val.to(device, non_blocking=True)
            
            # Update architecture parameters
            arch_loss, entropy = architect.step(x_val, y_val)
            
            # Clear cache periodically to prevent fragmentation
            if step % 10 == 0:
                torch.cuda.empty_cache()
            
            # Update network weights
            w_optimizer.zero_grad()
            logits = model(x_train)
            loss = F.cross_entropy(logits, y_train)
            loss.backward()
            nn.utils.clip_grad_norm_(
                [p for n, p in model.named_parameters() if 'alphas' not in n], 
                5.0
            )
            w_optimizer.step()
            
            train_loss += loss.item()
            arch_loss_sum += arch_loss
            entropy_sum += entropy
            
            pbar.set_postfix({
                'loss': f'{loss.item():.3f}',
                'arch_loss': f'{arch_loss:.3f}',
                'entropy': f'{entropy:.3f}'
            })
            
            # Delete variables to free memory
            del x_train, y_train, x_val, y_val, logits, loss
        
        scheduler.step()
        
        # Clear cache after each epoch
        torch.cuda.empty_cache()
        
        # Print epoch statistics
        avg_train_loss = train_loss / len(train_loader)
        avg_arch_loss = arch_loss_sum / len(train_loader)
        avg_entropy = entropy_sum / len(train_loader)
        
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"  Train Loss: {avg_train_loss:.4f}")
        print(f"  Arch Loss: {avg_arch_loss:.4f}")
        print(f"  Entropy: {avg_entropy:.4f}")
        
        # Save alpha checkpoint
        np.save(f"alphas_epoch{epoch}.npy", model.alphas.detach().cpu().numpy())
        
        # Show current genotype
        genotype = model.genotype()
        print(f"  Current Genotype: {genotype}")
        print()
    
    return model

# ==================== MAIN SEARCH PROCEDURE ====================
def main_search():
    """Main search procedure"""
    
    # Clean up checkpoint folders
    for base in [TRAIN_DIR, TEST_DIR]:
        chk = os.path.join(base, ".ipynb_checkpoints")
        if os.path.isdir(chk):
            print(f"Removing {chk}")
            shutil.rmtree(chk)
    
    # Load datasets
    print("Loading datasets...")
    full_train_ds = PancreaticDataset(TRAIN_DIR, img_size=224, augment=True)
    test_ds = PancreaticDataset(TEST_DIR, class_to_idx=full_train_ds.class_to_idx, 
                                img_size=224, augment=False)
    
    print(f"Class mapping: {full_train_ds.class_to_idx}")
    print(f"Total training samples: {len(full_train_ds)}")
    print(f"Total test samples: {len(test_ds)}")
    
    # Split train into train/val
    train_len = int(0.8 * len(full_train_ds))
    val_len = len(full_train_ds) - train_len
    train_ds, val_ds = random_split(
        full_train_ds, [train_len, val_len],
        generator=torch.Generator().manual_seed(SEED)
    )
    
    print(f"Search train: {len(train_ds)}, Search val: {len(val_ds)}")
    
    # Create data loaders with reduced batch size for memory efficiency
    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True,  # Reduced from 32
                             num_workers=2, pin_memory=True, persistent_workers=True)
    val_loader = DataLoader(val_ds, batch_size=16, shuffle=True,  # Reduced from 32
                           num_workers=2, pin_memory=True, persistent_workers=True)
    test_loader = DataLoader(test_ds, batch_size=16, shuffle=False,  # Reduced from 32
                            num_workers=2, pin_memory=True, persistent_workers=True)
    
    # Create model with smaller configuration for memory efficiency
    print("\nCreating search network...")
    model = SearchNetwork(
        C=12,           # Reduced from 16 (saves ~40% memory)
        num_cells=4,    # Reduced from 6 (saves memory)
        num_nodes=3,    # Reduced from 4 (fewer edges)
        num_classes=len(full_train_ds.class_to_idx)
    ).to(device)
    
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # Create architect
    arch_args = {
        'arch_lr': 3e-4,
        'arch_weight_decay': 1e-3
    }
    architect = Architect(model, arch_args)
    
    # Run search with reduced epochs for memory efficiency
    print("\nStarting architecture search...")
    model = train_search(model, train_loader, val_loader, architect, epochs=10)  # Reduced from 15
    
    # Get final genotype
    final_genotype = model.genotype()
    print("\n" + "="*50)
    print("FINAL GENOTYPE:")
    print(final_genotype)
    print("="*50)
    
    # Save results
    np.save("final_alphas.npy", model.alphas.detach().cpu().numpy())
    with open("genotype.txt", "w") as f:
        f.write(",".join(final_genotype))
    torch.save(model.state_dict(), "search_model.pth")
    
    print("\nSaved: final_alphas.npy, genotype.txt, search_model.pth")
    
    return final_genotype, full_train_ds.class_to_idx

# ==================== RUN ====================
if __name__ == "__main__":
    genotype, class_to_idx = main_search()

✓ Dataset found at ./dataset
Device: cuda
GPU: NVIDIA H100 80GB HBM3
GPU Memory: 85.03 GB
Loading datasets...
Class mapping: {'normal': 0, 'pancreatic_tumor': 1}
Total training samples: 999
Total test samples: 412
Search train: 799, Search val: 200

Creating search network...
Model parameters: 76,460

Starting architecture search...


Epoch 1/10: 100%|██████████| 50/50 [00:47<00:00,  1.04it/s, loss=0.569, arch_loss=0.562, entropy=1.792]



Epoch 1 Summary:
  Train Loss: 0.5464
  Arch Loss: 0.5593
  Entropy: 1.7918
  Current Genotype: ['dil_conv_3x3', 'sep_conv_5x5', 'sep_conv_3x3', 'dil_conv_3x3', 'sep_conv_3x3', 'dil_conv_3x3', 'dil_conv_3x3', 'dil_conv_3x3', 'sep_conv_5x5']



Epoch 2/10: 100%|██████████| 50/50 [00:46<00:00,  1.07it/s, loss=0.620, arch_loss=0.431, entropy=1.792]



Epoch 2 Summary:
  Train Loss: 0.5308
  Arch Loss: 0.5291
  Entropy: 1.7918
  Current Genotype: ['sep_conv_3x3', 'sep_conv_5x5', 'sep_conv_5x5', 'dil_conv_3x3', 'dil_conv_3x3', 'dil_conv_3x3', 'dil_conv_3x3', 'dil_conv_3x3', 'sep_conv_5x5']



...


Epoch 8 Summary:
  Train Loss: 0.4556
  Arch Loss: 0.4339
  Entropy: 1.7916
  Current Genotype: ['sep_conv_5x5', 'dil_conv_3x3', 'dil_conv_3x3', 'dil_conv_3x3', 'avg_pool_3x3', 'sep_conv_3x3', 'avg_pool_3x3', 'avg_pool_3x3', 'dil_conv_3x3']



Epoch 9/10: 100%|██████████| 50/50 [00:46<00:00,  1.07it/s, loss=0.400, arch_loss=0.543, entropy=1.792]



Epoch 9 Summary:
  Train Loss: 0.4393
  Arch Loss: 0.4308
  Entropy: 1.7916
  Current Genotype: ['sep_conv_3x3', 'sep_conv_5x5', 'avg_pool_3x3', 'dil_conv_3x3', 'avg_pool_3x3', 'sep_conv_3x3', 'avg_pool_3x3', 'avg_pool_3x3', 'dil_conv_3x3']



Epoch 10/10: 100%|██████████| 50/50 [00:46<00:00,  1.07it/s, loss=0.584, arch_loss=0.487, entropy=1.792]



Epoch 10 Summary:
  Train Loss: 0.4390
  Arch Loss: 0.4225
  Entropy: 1.7916
  Current Genotype: ['sep_conv_5x5', 'dil_conv_3x3', 'avg_pool_3x3', 'dil_conv_3x3', 'avg_pool_3x3', 'sep_conv_3x3', 'avg_pool_3x3', 'avg_pool_3x3', 'dil_conv_3x3']


FINAL GENOTYPE:
['sep_conv_5x5', 'dil_conv_3x3', 'avg_pool_3x3', 'dil_conv_3x3', 'avg_pool_3x3', 'sep_conv_3x3', 'avg_pool_3x3', 'avg_pool_3x3', 'dil_conv_3x3']

Saved: final_alphas.npy, genotype.txt, search_model.pth
